# Prep OPERA RTC CalVal data: stage 1, part 3

**Alex Lewandowski; Alaska Satellite Facility, University of Alaska Fairbanks**

## Performs additional data preparation after MGRS subsetting for OPERA RTC calibration and validation

**Notebook Requires**
- MGRS tiles of prepared OPERA RTC CalVal data created with Prep_OPERA_RTC_CalVal_data_stage1_part2.ipynb

**Actions**
- convert incidence angle maps from radians to degrees
- Use the Copernicus land cover data and the layover-shadow mask to create a mask of forested, valid pixels
- Determine foreslope, backslope, and flat areas by subtracting the ellipsoidal incidence angles from local incidence angles
- create 3 backscatter geotiffs for each polarity containing valid, forested pixels in:
    - foreslope regions
    - backslope regions
    - flat regions

In [ ]:
from ipyfilechooser import FileChooser
import math
import numpy as np
from pathlib import Path
import shutil

import numpy.ma as ma
from osgeo import gdal

In [ ]:
print("Select the directory holding your MGRS tile sub-directories")
fc = FileChooser(Path.cwd())
display(fc)

## Glob the MGRS tile subdirectories

In [ ]:
data_dir = Path(fc.selected_path)
mgrs = list()
for p in Path(data_dir).iterdir():
    if p.is_dir():
        mgrs.append(p)
mgrs

---
## Mask out non-forested land cover types

### Landcover Classifications

**Non-Forest**
- 0: No input data
- 20: Shrubs
- 30: Herbaceous vegetation
- 40: Cultivated and managed vegetation/agriculture (cropland)
- 50: Urban / built up
- 60: Bare / sparse vegetation
- 70: Snow and ice
- 80: Permanent water bodies
- 90: Herbaceous wetland
- 100: Moss and lichen
- 200: Open sea
- 255: missing values

**Forest**
- 111: Closed forest, evergreen needle leaf
- 112: Closed forest, evergreen, broad leaf
- 113: Closed forest, deciduous needle leaf
- 114: Closed forest, deciduous broad leaf
- 115: Closed forest, mixed
- 116: Closed forest, unknown
- 121: Open forest, evergreen needle leaf
- 122: Open forest, evergreen broad leaf
- 123: Open forest, deciduous needle leaf
- 124: Open forest, deciduous broad leaf
- 125: Open forest, mixed
- 126: Open forest, unknown

## mask invalid pixels (as determined from layover-shadow mask)

## Create 3 backscatter tiffs for each polarization:

- foreslope backscatter
- backslope backscatter
- flat backscatter 

Remove non-forest and invalid (layover/shadow) pixels in all output backscatter tiffs 

In [ ]:
for m in mgrs:
    local_inc = list(m.glob('*_lc_inc_map_*.tif'))[0]
    ellipse_inc = list(m.glob('*_ell_inc_map_*.tif'))[0]
    ls_map = list(m.glob('*_ls_map_*.tif'))[0]
    vh = list(m.glob("*_VH_*.tif"))[0]
    vv = list(m.glob("*_VV_*.tif"))[0]
    dem = list(m.glob("*_dem_*.tif"))[0]
    landcover = list(m.glob("*Discrete-Classification-map_*.tif"))[0]
    
    # Convert incidence angle maps to degrees
    to_deg = [local_inc, ellipse_inc]
    for i, ds in enumerate(to_deg):
        out = ds.parent/f"{ds.stem}_deg.tif"
        shutil.copy(ds, out)
        f = gdal.Open(str(out), gdal.GA_Update)
        array = f.ReadAsArray()

        array = array / math.pi * 180

        f.GetRasterBand(1).WriteArray(array)
        f.FlushCache()
        if i == 0:
            local_inc = out
        else:
            elipse_inc = out
        # ds.unlink()
        
    valid_forest_landcover = landcover.parent/f"{landcover.stem}_valid_forest.tif"
    shutil.copy(landcover, valid_forest_landcover)

    f_landcover = gdal.Open(str(valid_forest_landcover), gdal.GA_Update)
    landcover_ar = f_landcover.ReadAsArray()

    # Set all non-forest pixels to 0
    non_forest_types = [20, 30, 40, 50, 60, 70, 80, 90, 100, 200, 255]
    for val in non_forest_types:
        land_cover_masked = ma.masked_values(landcover_ar, val)
        landcover_ar = land_cover_masked.filled(fill_value=0)

    # set all forest pixels to 1
    forest_types = [111, 112, 113, 114, 115, 116, 121, 122, 123, 124, 125, 126]
    for val in forest_types:
        land_cover_masked = ma.masked_values(landcover_ar, val)
        landcover_ar = land_cover_masked.filled(fill_value=1)

    # set all pixels affected by layover or shadow to 0
    f_ls_mask = gdal.Open(str(ls_map))
    ls_array = f_ls_mask.ReadAsArray()
    ls_mask = ma.masked_where(ls_array!=1, ls_array)
    ls_array = ls_mask.filled(fill_value=0)

    # Update forest mask with invalid pixels from layover shadow mask
    ls_mask = ma.masked_where(ls_array==0, landcover_ar)
    landcover_ar = ls_mask.filled(fill_value=0)

    # Save valid_forest mask, containing values=1 for forested pixels unaffected by layover and/or shadow
    f_landcover.GetRasterBand(1).WriteArray(landcover_ar)
    f_landcover.FlushCache()
    
    # Subtract elliposidal incidence angles from local incidence angles
    local = gdal.Open(str(local_inc))
    local_ar = local.ReadAsArray()
    ellipse = gdal.Open(str(elipse_inc))
    ellipse_ar = ellipse.ReadAsArray()
    ell_minus_local_inc_arr = local_ar - ellipse_ar

    # Create foreslope, backslope, and flat backscatter geotiffs for each polarization
    for pol in [vh, vv]:
        foreslope = pol.parent/f"{pol.stem}_foreslope.tif"
        backslope = pol.parent/f"{pol.stem}_backslope.tif"
        flat = pol.parent/f"{pol.stem}_flat.tif"

        slopes = [foreslope, backslope, flat]
        for s in slopes:    
            shutil.copy(pol, s)
            f = gdal.Open(str(s), gdal.GA_Update)
            pol_array = f.ReadAsArray()

            flat_threshold = [-2, 2]
            
            if "backslope" in str(s):
                mask = ma.masked_where(ell_minus_local_inc_arr<=2, pol_array)
                # Replace values <= upper flat_threshold with 0
                masked_array = mask.filled(fill_value=0)
            elif "foreslope" in str(s):
                # Replace values >= lower flat_threshold with 0
                mask = ma.masked_where(ell_minus_local_inc_arr>=-2, pol_array)
                masked_array = mask.filled(fill_value=0)
            else:
                # Replace values outside of flat_threshold with 0
                mask = ma.masked_where(ell_minus_local_inc_arr>2, pol_array)
                masked_array = mask.filled(fill_value=0)
                mask = ma.masked_where(ell_minus_local_inc_arr<-2, pol_array)
                masked_array = mask.filled(fill_value=0)

            # Remove non-forest and invalid pixels
            invalid_mask = ma.masked_where(landcover_ar==0, masked_array)
            masked_array = invalid_mask.filled(fill_value=0)

            f.GetRasterBand(1).WriteArray(masked_array)
            f.FlushCache()

*Prep_OPERA_RTC_CalVal_data_stage1_part3 - Version 0.1.0 - April 2022*